In [ ]:
from eli5.lime import TextExplainer
from sklearn.preprocessing import FunctionTransformer
from lime.lime_text import LimeTextExplainer
import numpy as np
import texthero as hero
from texthero import preprocessing
import pandas as pd
from sklearn.pipeline import make_pipeline

In [ ]:
""":arg
x_test -- > text (without tfidf)
y_test --> test truth values
y_train --> train truth values
model -- > classifier such as rf, svm
vectorizer --> fitted tfidf vectorizer

"""
_text = x_test[x_test.index == 1820].tolist()
print('Text: ', _text)

_label = y_test[y_test.index == 1820].tolist()
print('Label: ', _label)

#classes = list(np.unique(y_train))
classes = le.classes_ # le is label encoder

def clean_text(text):
    text = pd.Series(text)

    # cleaning steps
    cleaning_pipeline = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_whitespace,
        preprocessing.remove_punctuation,
        preprocessing.remove_urls,
        preprocessing.remove_brackets,
        preprocessing.remove_stopwords,
        preprocessing.remove_digits,
        preprocessing.remove_angle_brackets,
        preprocessing.remove_curly_brackets,
        preprocessing.stem
        #preprocessing.tokenize
    ]

    # apply pipeline to text
    processed_text = text.pipe(hero.clean, cleaning_pipeline)

    return processed_text.tolist()

cleaned_txt = clean_text(_text[0])
print(cleaned_txt)

transformer = FunctionTransformer(clean_text)
txt_clf = make_pipeline(transformer, vectorizer,  model)
te = TextExplainer(random_state=42)
te.fit(cleaned_txt[0], txt_clf.predict_proba)
te.show_prediction(target_names=list(np.unique(classes)), top=10, preserve_density=True, highlight_spaces=True, horizontal_layout=True)

In [ ]:
explainer = LimeTextExplainer(class_names=classes)
exp = explainer.explain_instance(_text[0], txt_clf.predict_proba, num_features=10, labels=classes, top_labels=5)
exp.show_in_notebook(text=True)
exp.save_to_file('sample_explained.html')